# Grant Routes

In [ ]:
#| default_exp routes.grant

In [ ]:
#| exporti
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de


In [ ]:
#| export
class GetGrants_Error(de.DomoError):
    def __init__(self, status, message, domo_instance, function_name):
        super().__init__(status = status, message = message, domo_instance = domo_instance, function_name = function_name)


In [ ]:
#| hide
try:
    raise GetGrants_Error(status = 405, message = "error", domo_instance = 'test', function_name= 'GetData')

except Exception as e:
    print(e)

GetData: Status 405 - error at test


In [ ]:
# | export


async def get_grants(auth: dmda.DomoAuth,
                     debug_api: bool = False,
                     session: httpx.AsyncClient = None) -> rgd.ResponseGetData:

    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/authorities'

    res = await gd.get_data(auth=auth,
                            url=url,
                            method='GET',
                            debug_api=debug_api,
                            session=session)

    if not res.is_success:
        raise GetGrants_Error(status=res.status,
                              message=res.message,
                              domo_instance=auth.domo_instance,
                              function_name='get_grants')
    
    if len(res.response) == 0:
        raise GetGrants_Error(status=res.status,
                              message=f'{len(res.response)} grants returned',
                              domo_instance=auth.domo_instance,
                              function_name='get_grants')

    return res


#### sample implementation of get_grants

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

res = await get_grants(auth=token_auth)

pd.DataFrame(res.response[0:5])

,authority,title,description,authorityUIGroup,userCount,roleIds,dependsOnAuthorities,systemGroup
0,authorization.roles,Manage all roles,"Create, edit and delete custom roles. Assign a...",COMPANY,16,"[1, 275763436]",[ui.admin],True
1,audit,View activity logs,View and export audit logs.,COMPANY,113,"[1, 275763436, 1927158482, 2097317660]",[ui.admin],True
2,customer.edit,Manage all company settings,"Manage company metadata, authentication rules,...",COMPANY,16,"[1, 275763436]",[ui.admin],True
3,developer.token.manage,Manage all access tokens,Create and delete any access tokens.,COMPANY,21,"[1, 275763436, 1927158482]",[ui.admin],True
4,domo.system.messages,Receive Domo System Messages,Receive Domo system messages about maintenance...,COMPANY,21,"[1, 275763436, 1927158482]",[ui.admin],True


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
